# Ion exchange in dune sand and dunewater

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Mar 10th, 2022</p>

In this tutorial, we calculate the exchangeable cations in dune sand and dunewater. We consider 6 g of dune sand with 10 meq/kg of CEC (cation exchange capacity), which results in 0.06 mol of exchanger X (60 meq/L, default 1L).

Our goal is to reproduce the results of the PHREEQC script below.

```text
DATABASE ../database/phreeqc.dat
TITLE Exchange in Dune sand

SOLUTION 1
pressure 1.0
temperature 25.0
Na 1.10006
Mg 0.48
Ca 1.9

EXCHANGE 1
X 0.06	#moles
-equilibrate 1
```

We start with setting up the chemical system and other means of modelling ion-exchange in Reaktoro. For that, we use the PHREEQC database phreeqc.dat that contains ion-exchange species, their corresponding reactions, and necessary parameters.

In [1]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O C Ca Na Mg Cl"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

The ion-exchange phase is modelled by the {{IonExchangePhase}} class. We select only three ion exchange species: NaX, CaX{{_2}}, and MgX{{_2}}, which can be formed from major ions Na<sup>+</sup>, Ca<sup>2+</sup>, and Mg<sup>2+</sup> in the dunewater. We use the Gaines-Thomas activity model.

In [2]:
exchange = IonExchangePhase("NaX CaX2 MgX2")
exchange.setActivityModel(ActivityModelIonExchangeGainesThomas())

system = ChemicalSystem(db, solution, exchange)

# Define the equilibrium solver
solver = EquilibriumSolver(system)

Below, we define the initial chemical state for the ion-exchange calculation involving dune sand and dunewater at 25 °C and 1 bar.

```{note}
We note that considered below initial amount of NaX is set to 0.06 * 1e-6 mol. This way we imitate an unlimited source of brine in the equilibrium calculations. Therefore, we can expect the order difference in the amount of precipitated ion exchange species.
```

```{note}
Unlike PHREEQC, to model the ion exchange in Reaktoro we always assume that the exchanger (X, in this case) is occupied by some ions.
```

In [3]:
# Define initial equilibrium state
state = ChemicalState(system)
state.setTemperature(25, "celsius")
state.setPressure(1.0, "bar")
# Scale solution recipe to match the values of the PHREEQC examples
state.set("H2O"   , 1.00, "kg")
state.set("Na+" , 1.10, "mmol")
state.set("Mg+2", 0.48, "mmol")
state.set("Ca+2", 1.90, "mmol")
# Set the number of exchange assuming that it is completely occupied by sodium
state.set("NaX" , 0.06, "umol")

We expect the following set of ion exchange reactions happening between dunewater and exchanger site X<sup>-</sup> (occupied with sodium ion):

$$
\begin{alignat}{4}
\tfrac{1}{2} {\rm Ca^{+2}} + {\rm NaX} & \rightleftharpoons {\rm Na}^+ + \tfrac{1}{2} {\rm CaX}_2\\
\tfrac{1}{2} {\rm Mg^{+2}} + {\rm NaX} & \rightleftharpoons {\rm Na}^+ + \tfrac{1}{2} {\rm MgX}_2\\
\end{alignat}
$$

After the initial chemical state is defined, we can create the equilibrium solver that will perform a Gibbs energy minimization calculation to find the equilibrium state:

In [4]:
solver = EquilibriumSolver(system)
solver.solve(state)

By printing the instance `state`, we can inspect the content of calculated equilibrium state:

In [5]:
print(state)

+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   298.1500 |    K |
| Pressure        |     1.0000 |  bar |
| Charge:         | 5.8600e+03 |  mol |
| Element Amount: |            |      |
| :: X            | 6.0000e-02 |  mol |
| :: H            | 1.1101e+08 |  mol |
| :: C            | 1.2000e-15 |  mol |
| :: O            | 5.5506e+07 |  mol |
| :: Na           | 1.1001e+03 |  mol |
| :: Mg           | 4.8000e+02 |  mol |
| :: Cl           | 1.0000e-16 |  mol |
| :: Ca           | 1.9000e+03 |  mol |
| Species Amount: |            |      |
| :: CO3-2        | 1.0000e-16 |  mol |
| :: H+           | 1.1653e-01 |  mol |
| :: H2O          | 5.5506e+07 |  mol |
| :: CO2          | 1.0000e-16 |  mol |
| :: (CO2)2       | 1.0000e-16 |  mol |
| :: HCO3-        | 1.0000e-16 |  mol |
| :: CH4          | 1.0000e-16 |  mol |
| :: Ca+2         | 1.9000e+03 |  mol |
| :: CaCO3        | 1.0000e-16 |  mol |


We can now compute the properties of the brine solution after the ion exchange has been carried out. For this purpose, the {{AqueousProps}} class is used.

In [6]:
aqprops = AqueousProps(state)
print("I  = %f mol/kgw" % float(aqprops.ionicStrength()))
print("pH = %f" % float(aqprops.pH()))
print("pE = %f" % float(aqprops.pE()))


I  = 0.005310 mol/kgw
pH = 6.966945
pE = -3.999323


To inspect the ion exchange properties of species NaX, CaX{{_2}}, and MgX{{_2}}, the class {{IonExchangeProps}} can be used, which is done next:

In [7]:
exprops = IonExchangeProps(state)
print(exprops)

+--------------------------------------+------------+------+
| Property                             |      Value | Unit |
+--------------------------------------+------------+------+
| Element Amounts:                     |            |      |
| :: X                                 | 6.0000e-02 |  mol |
| :: Na                                | 5.5828e-04 |  mol |
| :: Mg                                | 4.0703e-03 |  mol |
| :: Ca                                | 2.5651e-02 |  mol |
| Species Amounts:                     |            |      |
| :: NaX                               | 5.5828e-04 |  mol |
| :: CaX2                              | 2.5651e-02 |  mol |
| :: MgX2                              | 4.0703e-03 |  mol |
| Equivalents:                         |            |      |
| :: NaX                               |     0.0006 |   eq |
| :: CaX2                              |     0.0513 |   eq |
| :: MgX2                              |     0.0081 |   eq |
| Equivalent Fractions: 